# Tutorial pyOSRD #6
Creating custom agent

In [4]:
from pyosrd.agents.scheduler_agent import SchedulerAgent
import pandas as pd
from dataclasses import dataclass

## Creating a dumb agent
Create an agent that add 60 seconds of delay at the second zone the first train go throught.

In [5]:

class DumbSchedulerAgent(SchedulerAgent):
    @property
    def steps_extra_delays(self) -> pd.DataFrame:
        """As an example, make train1 stop 60s more in second zone"""
        extra_delays = self.ref_schedule.durations * 0.
        extra_delays.iloc[0, 0] = 60.
        return extra_delays


## Test our agent on all scenarii
We can launch our custom agent on all scenarii implemented in the module OSRD.with_delays.

In [6]:
from pyosrd.agents.scheduler_agent import regulate_scenarii_with_agents
from functools import partial
from pyosrd.schedules.weights import all_steps

agent = DumbSchedulerAgent("dumb")


regulate_scenarii_with_agents("all", agent)

[10:33:47,995] [INFO]  [StandaloneSimulationCommand] Loading infra: ./infra.json
[10:33:48,025] [INFO]              [FullInfra] parsing infra
[10:33:48,088] [INFO]              [FullInfra] adaptation to kotlin
[10:33:48,109] [INFO]              [FullInfra] loading signals
[10:33:48,115] [INFO]              [FullInfra] building blocks
[10:33:48,121] [INFO]  [StandaloneSimulationCommand] Loading input: ./simulation.json
[10:33:48,280] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.15
[10:33:48,364] [DEBUG]             [Allowance] computing range n°1
[10:33:48,366] [DEBUG]             [Allowance]   computing section n°1
[10:33:48,366] [DEBUG]             [Allowance]   target time = 310.84094999999996
[10:33:48,366] [DEBUG]             [Allowance]     starting attempt 1
[10:33:48,367] [DEBUG]             [Allowance]     envelope time 518.958
[10:33:48,368] [DEBUG]             [Allowance]     starting attempt 2
[10:33:48,368] [DEBUG]             [Allowanc

,dumb
c1_2trains_delay_train1,360.000000
c1y2_2trains_conflict,120.000000
c1y2_2trains_no_conflict,219.637177
c2y11s_conflict_20_trains,635.245468
c2y1y2y_2_trains_no_conflict,310.000000
c2y1y2y_2_trains_reorder,450.000000
c2y1y2y_2_trains_same_order,375.000000
multistation_multitrains_randomdelay,120.000000


## Compare two agents

The function regulate_scenarii_with_agents can be launched on multiple agents at the same time. It returns a pandas DataFrame containing the ponderated delays
for all scenarii.

An agent can be compared to another one using those scenarii.

In [7]:

class DumberSchedulerAgent(SchedulerAgent):
    @property
    def steps_extra_delays(self) -> pd.DataFrame:
        """As an example, make train1 stop 120s more in second zone"""
        extra_delays = self.ref_schedule.durations * 0.
        extra_delays.iloc[0, 1] = 120.
        return extra_delays

To speed up the notebook execution the next call is done only on one scenario "c1_delay".

In [8]:

agent_dumber = DumberSchedulerAgent("dumber")
regulate_scenarii_with_agents("c1_2trains_delay_train1", [agent, agent_dumber])

[10:34:02,186] [INFO]  [StandaloneSimulationCommand] Loading infra: ./infra.json
[10:34:02,216] [INFO]              [FullInfra] parsing infra
[10:34:02,287] [INFO]              [FullInfra] adaptation to kotlin
[10:34:02,315] [INFO]              [FullInfra] loading signals
[10:34:02,322] [INFO]              [FullInfra] building blocks
[10:34:02,328] [INFO]  [StandaloneSimulationCommand] Loading input: ./simulation.json
[10:34:02,507] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.30
[10:34:02,584] [DEBUG]             [Allowance] computing range n°1
[10:34:02,586] [DEBUG]             [Allowance]   computing section n°1
[10:34:02,587] [DEBUG]             [Allowance]   target time = 310.84094999999996
[10:34:02,587] [DEBUG]             [Allowance]     starting attempt 1
[10:34:02,588] [DEBUG]             [Allowance]     envelope time 518.958
[10:34:02,588] [DEBUG]             [Allowance]     starting attempt 2
[10:34:02,588] [DEBUG]             [Allowanc

,dumb,dumber
c1_2trains_delay_train1,360.0,480.0
